In [10]:
import os
from random import randint, uniform
import re
import numpy as np
import wave
import contextlib


## Calculating recordings lengths

In [24]:

total = 0
corrupted=0
files=0
for directory in os.listdir("./sections/"):
    if os.path.isdir("./sections/"+directory):
        for file in os.listdir("./sections/"+directory):
            if file.endswith(".wav"):
                files+=1
                fname = "./sections/"+directory+"/"+file
                try:
                    with contextlib.closing(wave.open(fname,'r')) as f:
                        frames = f.getnframes()
                        rate = f.getframerate()
                        duration = frames / float(rate)
                        total+=duration
                except Exception as e:
                    corrupted+=1

print("We have a total of ",total," seconds",", ",total/(60*60)," hours")

We have a total of  8069.399249999986  seconds ,  2.2414997916666626  hours


In [25]:
print('This is the whole files:',files)
print('The corrupted ones are:' , corrupted)

This is the whole files: 1149
The corrupted ones are: 88


# Organizing data

### spliting records to train, val and test sets

In [75]:
!mkdir data
!mkdir data/records

In [79]:
# Extracting all the non-corrupted files
wav_files = []
all_linkers  = dict()
set_files_found = set()
for directory in os.listdir("./sections/"): # parent directory of the recordings, it should contain folders that contain wav,json and txt files
    if os.path.isdir("./sections/"+directory):
        to_remove = []
        linker = []
        linker_data = open("./sections/"+directory+"/"+[file for file in os.listdir("./sections/"+directory) if file.endswith(".txt")][0]).readlines()
        text_file_name = linker_data[0].split(" ")[0].split("/")[-1]
        
        
        for file in os.listdir("./sections/"+directory):
#             if file.endswith(".txt"):
#                 linker_data = open("./sections/"+directory+"/"+file).readlines()
                
            if file.endswith(".wav"):
                if file in set_files_found:
                    continue
                set_files_found.add(file)
                fname = "./sections/"+directory+"/"+file
                try:
                    with contextlib.closing(wave.open(fname,'r')) as f:
                        frames = f.getnframes()
                        rate = f.getframerate()
                        duration = frames / float(rate)
                        wav_files.append(fname)
                except Exception as e:
                    to_remove.append(file)
        for file in to_remove:
            i=0
            while i<len(linker_data):
                if file in linker_data[i]:
                    linker_data.pop(i)
                i+=1
        linker.extend(linker_data)
        all_linkers[text_file_name] = linker
        

            

In [80]:
all_linkers.keys()

dict_keys(['section5.txt', 'section10.txt', 'section9.txt', 'section4.txt', 'section1.txt', 'sec11.txt', 'section26.txt', 'section8.txt', 'section13.txt', 'section12.txt', 'section7.txt', 'section17.txt', 'section15.txt', 'section6.txt'])

In [55]:
# linkers = {key:set(linker) for key,linker in all_linkers.items()}
# sets = list(linkers.values())
# for linker in sets:
#     for i  in range(len(sets)):
#         for j in range(i+1,len(sets)):
#             sets[i] = sets[i].difference(sets[j])
# for key,value in all_linkers.items():
#     all_linkers[key] = linkers[key]
# sum([len(linker) for linker in all_linkers.values()])

In [81]:
print("After removing the corrupted files, we have {} .wav files left. " . format(len(wav_files)))

After removing the corrupted files, we have 830 .wav files left. 


In [82]:
to_copy = " ".join(wav_files)
!cp -t data/records/ {to_copy}

In [83]:
for section,linker in all_linkers.items():
    for i,link in enumerate(linker):
        line = link.split(";")[0].split("(")[1].split(")")[0].strip()
        wav = link.split(";")[1].strip().split("/")[-1]
        linker[i] = wav+":"+line


In [59]:
# out_linker = open("./data/linker.txt","w")
# out_linker.write("\n".join(linker))
# out_linker.close()

In [84]:
!mkdir ./data/records/train
!mkdir ./data/records/test
!mkdir ./data/records/val


In [85]:
import numpy as np
np.random.seed(0)
indices = np.random.permutation(len(wav_files))

test_idx = indices[:len(indices)//2]
validation_portion = int(len(indices)//2 * 0.2)
train_idx = indices[len(indices)//2:-validation_portion]
valid_idx = indices[-validation_portion:]

In [86]:
train_set_files = [wav_files[i].split("/")[-1] for i in train_idx]
valid_set_files = [wav_files[i].split("/")[-1] for i in valid_idx]
test_set_files = [wav_files[i].split("/")[-1] for i in test_idx]

In [87]:
to_copy_train = "./data/records/" + " ./data/records/".join(train_set_files)
to_copy_valid = "./data/records/" + " ./data/records/".join(valid_set_files)
to_copy_test = "./data/records/" + " ./data/records/".join(test_set_files)

In [88]:
!mv -t data/records/train/ {to_copy_train}
!mv -t data/records/val/ {to_copy_valid}
!mv -t data/records/test/ {to_copy_test}

### making of chars.txt file

In [95]:
import re
chars = {" ":1,"ε":0}
text_data = []
char_idx = 2
for section_id,linker in all_linkers.items():
    #section_id = re.sub("_","",section_id)+".txt"
    text_file = open("./txt/"+section_id,"r").readlines() # Modify this to the directory of your txt file that you recorded with
    
    for link in linker:
        
        file,idx = link.split(":")[0],int(link.split(":")[1].split(" ")[1])-1
        line = text_file[idx]
        print(section_id,link)
        if len(line.strip())<=3:
            continue
        line = line.split("##")[0].strip()
        line = re.sub("[\[\]|٪%«»_?’”!,;\)\(:.’‘-]","",line) # Clean unnecessary characters from the data, this is for arabic
        line = re.sub("[—\u200b]","",line) # Clean unnecessary characters from the data, this is for arabic
        text_data.append((line,file.split(".")[0]))
        char_set = set(line)
        for c in char_set:
            if c not in chars:
                chars[c]=char_idx
                char_idx+=1
     
    



section5.txt 200703-143752_pcm_65e_elicit_0.wav:line 1
section5.txt 200703-143752_pcm_65e_elicit_1.wav:line 2
section5.txt 200703-143752_pcm_65e_elicit_2.wav:line 3
section5.txt 200703-143752_pcm_65e_elicit_3.wav:line 4
section5.txt 200703-143752_pcm_65e_elicit_4.wav:line 5
section5.txt 200703-143752_pcm_65e_elicit_5.wav:line 6
section5.txt 200703-143752_pcm_65e_elicit_6.wav:line 7
section5.txt 200703-143752_pcm_65e_elicit_7.wav:line 8
section5.txt 200703-143752_pcm_65e_elicit_8.wav:line 9
section5.txt 200703-143752_pcm_65e_elicit_9.wav:line 10
section5.txt 200703-143752_pcm_65e_elicit_10.wav:line 11
section5.txt 200703-143752_pcm_65e_elicit_11.wav:line 12
section5.txt 200703-143752_pcm_65e_elicit_12.wav:line 13
section5.txt 200703-143752_pcm_65e_elicit_13.wav:line 14
section5.txt 200703-143752_pcm_65e_elicit_14.wav:line 15
section5.txt 200703-143752_pcm_65e_elicit_15.wav:line 16
section5.txt 200703-143752_pcm_65e_elicit_16.wav:line 17
section5.txt 200703-143752_pcm_65e_elicit_17.wav:l

In [96]:
raw_text = "\n".join([wav+":"+line for line,wav in text_data])
with open("data/raw_text_file.txt","w") as f:
    f.write(raw_text)


In [97]:
indices_text = []
for line,wav in text_data:
    line = list(line)
    indicies = []
    for c in line:
        indicies.append(str(chars[c]))
    indices_text.append(wav+" "+" ".join(indicies))


In [98]:
len(chars)

61

In [99]:
chars

{' ': 1,
 'ε': 0,
 'p': 2,
 'a': 3,
 'b': 4,
 's': 5,
 'l': 6,
 'e': 7,
 'm': 8,
 'r': 9,
 'g': 10,
 't': 11,
 'S': 12,
 'd': 13,
 'u': 14,
 'i': 15,
 'n': 16,
 'f': 17,
 'o': 18,
 'B': 19,
 'y': 20,
 'v': 21,
 'c': 22,
 'w': 23,
 'I': 24,
 'h': 25,
 'k': 26,
 'x': 27,
 'U': 28,
 'T': 29,
 'O': 30,
 'H': 31,
 'A': 32,
 'Y': 33,
 'E': 34,
 'M': 35,
 'D': 36,
 'N': 37,
 'W': 38,
 'j': 39,
 'J': 40,
 '3': 41,
 '0': 42,
 'P': 43,
 '1': 44,
 '4': 45,
 'G': 46,
 'q': 47,
 'F': 48,
 'L': 49,
 'K': 50,
 '“': 51,
 '2': 52,
 'C': 53,
 'z': 54,
 'R': 55,
 'Q': 56,
 'V': 57,
 '6': 58,
 '9': 59,
 'Z': 60}

In [100]:
indicies_text = "\n".join(indices_text)
with open("data/chars.txt","w") as f:
    f.write(indicies_text)

In [101]:
with open("data/charset.txt","w") as js:
    js.write(str(chars))

In [102]:
with open("data/charset.txt") as js:
    charset = eval(js.read())

In [103]:
charset

{' ': 1,
 'ε': 0,
 'p': 2,
 'a': 3,
 'b': 4,
 's': 5,
 'l': 6,
 'e': 7,
 'm': 8,
 'r': 9,
 'g': 10,
 't': 11,
 'S': 12,
 'd': 13,
 'u': 14,
 'i': 15,
 'n': 16,
 'f': 17,
 'o': 18,
 'B': 19,
 'y': 20,
 'v': 21,
 'c': 22,
 'w': 23,
 'I': 24,
 'h': 25,
 'k': 26,
 'x': 27,
 'U': 28,
 'T': 29,
 'O': 30,
 'H': 31,
 'A': 32,
 'Y': 33,
 'E': 34,
 'M': 35,
 'D': 36,
 'N': 37,
 'W': 38,
 'j': 39,
 'J': 40,
 '3': 41,
 '0': 42,
 'P': 43,
 '1': 44,
 '4': 45,
 'G': 46,
 'q': 47,
 'F': 48,
 'L': 49,
 'K': 50,
 '“': 51,
 '2': 52,
 'C': 53,
 'z': 54,
 'R': 55,
 'Q': 56,
 'V': 57,
 '6': 58,
 '9': 59,
 'Z': 60}

In [113]:
import os
file  = open("./data/chars.txt").readlines()
file_names = []
for line in file:
    file_names.append(line.split()[0])
    if (line.split())==1:
        print(line)

In [114]:
len(file_names)

829

In [119]:
c=0
not_found = []
for dir in os.listdir("./data/records/"):
    if os.path.isdir("./data/records/"+dir):
        for file in os.listdir("./data/records/"+dir):
            if file.split(".")[0] not in file_names:
                not_found.append("./data/records/" + dir+ "/" + file)
                c+=1
print(c)

0


In [117]:
not_found = " ".join(not_found)

In [109]:
!mkdir not_found

In [118]:
!mv -t not_found {not_found}